# Import Libraries

In [114]:
import nltk
from collections import Counter
from collections import defaultdict
import pandas as pd
import numpy as np
import re, string

# Create a function to Remove Punctuation and Clean Data

In [40]:
def remove_punctuation(n):
    return ''.join(x for x in n if x not in set(string.punctuation))

In [43]:
file = open("train.txt", encoding = "utf-8")
file1 = file.read()

In [45]:
file1_cleaned = remove_punctuation(re.sub("\n","", file1)).lower()
tokens = file1_cleaned.split()
train_one_tokens = Counter(tokens)

# Create tokens for the training

In [46]:
cells = len(train_one_tokens)
keys = list(train_one_tokens.keys())

# Create Dataframe or Training 

In [47]:
dataset = pd.DataFrame(np.zeros(cells*cells).reshape(cells,cells), columns=keys, index=keys)

In [48]:
dataset_gt = pd.DataFrame(np.zeros(cells*cells).reshape(cells,cells), columns=keys, index=keys)

In [49]:
dataset_kn = pd.DataFrame(np.zeros(cells*cells).reshape(cells,cells), columns=keys, index=keys)

In [50]:
dataset.head(5)

,aer,banknote,berlitz,calloway,centrust,cluett,fromstein,gitano,guterman,hydroquebec,...,lungcancer,bikers,bofors,parsow,caci,isi,chestman,tci,trecker,unilab
aer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
banknote,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
berlitz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
calloway,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
centrust,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Implementing Ngram Model (Bigram)

In [54]:
bigr_list = nltk.bigrams(file1_cleaned.split())

In [55]:
bi = []
for bigram in bigr_list:
    bi.append(bigram)
bi_dict = Counter(bi)

In [62]:
def ngram(keys, bigram_dict, one_tokens, df):
    #Getting the probability of each key
    for key1 in keys:
        for key2 in keys:
            if bigram_dict[(key1,key2)] > 0 and one_tokens[key1] > 0:
                key_prob = bigram_dict[(key1,key2)] / one_tokens[key1]
                dataset.at[key1,key2] = key_prob 
    return dataset

# Create Function for predicting words using Ngram

In [63]:
def predictions(dataset, keys):
    highest_prob = list(dataset.idxmax(axis=1))
        
        #Mapping keys to highest prob	
    predicted_values = defaultdict() 
    for i in range(len(highest_prob)):
        predicted_values.update({keys[i]:highest_prob[i]})
        
    return predicted_values

# Create function to print Predictions

In [64]:
  def print_predictions(predicted_values, name):
    for k, v in predicted_values.items():
            filename = f"predicted_{name}.txt"
        #if v  not in 'aer':
            f = open(filename,'a')
            f.write(f"{k} : {v}\n")
            #print(f"{k} : {v}")

In [66]:
dataset = ngram(keys, bi_dict, train_one_tokens, dataset)

In [67]:
dataset.at['floor','traders']

0.14018691588785046

In [68]:
predicted_values_us = predictions(dataset, keys)

In [69]:
print_predictions(predicted_values_us, "ngram") 

# Implement Good turing 

In [44]:
def good_turing(keys, bigram_dict, df):
    n_c = Counter(list(bigram_dict.values()))
    #print(n_c)
    length_corpus = len(bi)
    default_val = n_c[1] / length_corpus
    #print(default_val)
    
    for key1 in keys:
            for key2 in keys:
                og_count = bigram_dict[(key1,key2)]
                if og_count == 0:
                    dataset.at[key1, key2] = default_val
                else:
                    adjusted_count = ((og_count+1) * n_c[og_count+1]) / n_c[og_count]
                    gt_prob = adjusted_count / total_corpus
                    dataset.at[key1,key2] = gt_prob
    return dataset

In [45]:
dataset_gt = good_turing(keys, bigram_dict, dataset_gt)

Counter({1: 189376, 2: 36725, 3: 15425, 4: 8433, 5: 5376, 6: 3583, 7: 2711, 8: 1923, 9: 1520, 10: 1287, 11: 1103, 12: 844, 13: 712, 14: 626, 15: 495, 16: 470, 17: 389, 18: 323, 19: 307, 20: 257, 22: 255, 21: 246, 23: 223, 24: 202, 26: 174, 25: 171, 27: 145, 28: 141, 29: 124, 30: 119, 32: 94, 31: 94, 34: 87, 33: 80, 35: 71, 36: 69, 38: 64, 41: 58, 40: 56, 42: 54, 39: 52, 43: 52, 37: 51, 47: 50, 46: 47, 44: 45, 45: 45, 49: 44, 51: 39, 50: 34, 54: 34, 56: 34, 52: 33, 59: 31, 48: 30, 55: 25, 60: 25, 65: 24, 58: 24, 74: 23, 72: 23, 62: 22, 57: 22, 70: 21, 63: 21, 64: 21, 71: 19, 81: 19, 68: 18, 53: 18, 67: 18, 76: 17, 73: 16, 77: 16, 69: 16, 100: 15, 66: 15, 82: 14, 80: 13, 79: 13, 89: 12, 75: 12, 91: 12, 85: 12, 61: 11, 78: 11, 127: 10, 92: 10, 86: 10, 84: 9, 95: 9, 120: 9, 83: 9, 97: 8, 112: 8, 88: 8, 161: 8, 101: 8, 184: 7, 94: 7, 131: 7, 96: 7, 148: 6, 99: 6, 143: 6, 132: 6, 115: 6, 111: 6, 103: 6, 98: 6, 113: 6, 197: 5, 128: 5, 107: 5, 104: 5, 93: 5, 87: 5, 135: 5, 144: 5, 106: 5, 105:

In [49]:
dataset_gt.head(2)

,aer,banknote,berlitz,calloway,centrust,cluett,fromstein,gitano,guterman,hydroquebec,...,lungcancer,bikers,bofors,parsow,caci,isi,chestman,tci,trecker,unilab
aer,0.215669,4.417030e-07,2.156693e-01,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,...,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669
banknote,0.215669,2.156693e-01,4.417030e-07,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,...,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669,0.215669


In [50]:
predicted_values_gt = predictions(dataset_gt)

# Make predictions using Good Turing

In [51]:
def print_predictions(predicted_values)
    for k, v in predicted_values.items():
            f = open('predicted_gt.txt','a')
            f.write(f"{k} : {v}\n")

# Testing on valid.txt to check perplexity of the model 

In [72]:
valid_file = open("valid.txt", encoding = "utf-8")
file1_valid_data = valid_file.read()

In [73]:
file1_valid_data_cleaned = remove_punctuation(re.sub("\n","", file1_valid_data)).lower()
valid_tokens = file1_valid_data_cleaned.split()
valid_one_tokens = Counter(valid_tokens)

In [74]:
valid_cells = len(valid_one_tokens)
valid_keys = list(valid_one_tokens.keys())

In [75]:
#len(valid_keys)
dataset_valid = pd.DataFrame(np.zeros(valid_cells*valid_cells).reshape(valid_cells,valid_cells), columns=valid_keys, index=valid_keys)

In [76]:
dataset_valid.head(3)

,consumers,may,want,to,move,their,telephones,a,little,closer,...,tools,45yearold,spurring,milwaukee,reinvestment,controlling,satisfactory,rewards,patience,driver
consumers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
may,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
want,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
valid_bi_list = nltk.bigrams(file1_valid_data_cleaned.split())
valid_bigram = []
for bigram in valid_bi_list:
    valid_bigram.append(bigram)
valid_bi_dict = Counter(valid_bigram)

In [86]:
dataset_test = ngram(valid_keys, valid_bi_dict, valid_one_tokens, dataset_valid)

In [87]:
predicted_values_test = predictions(dataset_valid, valid_keys)
print_predictions(predicted_values_test,"test")

In [88]:
def ngram_predictions(predicted_values):
    predict = open('input.txt','r')
    lines = predict.readlines()
    input_test = lines[:30]
    for line in input_test:
        predicted_words_list = []
        line = re.sub("___", "",line.rstrip())
        line_list = line.split()
        last_word = line_list[-1]
        while predicted_values[last_word] is not None:
            pred = predicted_values[last_word]
            if pred == 'unk' or pred == 'n': break
            #print(pred)
            predicted_words_list.append(pred)
            last_word = pred
        
        print(f"{line}->{predicted_words_list}")
        

In [89]:
ngram_predictions(predicted_values_us)

but while the new york stock exchange did n't fall ->['in', 'the']
some circuit breakers installed after the october N crash failed ->['to']
the N stock specialist firms on the big board floor ->['traders', 'said', 'the']
big investment banks refused to step up to the plate ->['and']
heavy selling of standard & poor 's 500-stock index futures ->['prices', 'for', 'the']
seven big board stocks ual amr bankamerica walt disney capital ->['markets', 'in', 'the']
once again the specialists were not able to handle the ->[]
<unk> james <unk> chairman of specialists henderson brothers inc. it ->['s']
when the dollar is in a <unk> even central banks ->['and']
speculators are calling for a degree of liquidity that is ->['a']
many money managers and some traders had already left their ->[]
then in a <unk> plunge the dow jones industrials in ->['the']
<unk> trading accelerated to N million shares a record for ->['the']
at the end of the day N million shares were ->[]
the dow 's decline was second i

# K-Nesser Function to predict words

In [116]:
def k_nesser():
    d = 0.04
    for key1 in valid_keys:
                for key2 in valid_keys:
                    og_count = valid_bi_dict[(key1,key2)] 
                    single_token_count = valid_single_tokens[key2]
                    kn_prob = max((og_count - d), 0) / (d / single_token_count )
                    df.at[key1, key2] = kn_prob

# Implementing Perplexity score

In [115]:
prob = 0
for bigram in valid_bigram:
    key1, key2 = bigram
    if df.at[key1, key2] > 0:
            prob = prob + np.log(dataset_test.at[key1, key2])
ppr = np.exp(prob/ len(valid_bigram))
ppr

0.06853272882625158